In [9]:
import numpy as np

In [ ]:
def myEM(data, G, itmax,):
    

In [59]:
data = []

with open('faithful.dat') as faithful:
    for row in faithful:
        data.append(row.split()[1:])
        
    # convert to float values
    data = np.array([[float(value) for value in row] for row in data[1:]])


In [79]:
# Test G=2

G = 2
itmax = 20
x = data
n = len(x)
p1 = 10 / n
p2 = 1 - p1
cluster_1 = x[:10]
cluster_2 = x[10:]
mu1 = np.mean(cluster_1, axis=0)
mu2 = np.mean(cluster_2, axis=0)

cov_matrix_1 = np.sum([(x[0] - mu1).reshape(-1, 1) @ (x[0] - mu1).reshape(-1, 1).T for i in cluster_1], axis=0)
cov_matrix_2 = np.sum([(x[0] - mu2).reshape(-1, 1) @ (x[0] - mu2).reshape(-1, 1).T for i in cluster_2], axis=0)

# 2x2 matrix
cov_matrix = (cov_matrix_1 + cov_matrix_2) / n
print(cov_matrix)


[[1.38930519e-02 9.02925674e-01]
 [9.02925674e-01 6.56887741e+01]]


In [ ]:
prob, mean, Sigma, loglik_val = myEM()

print("prob")
print(prob)

print("mean")
print(mean)

print("Sigma")
print(Sigma)

print("loglik_val")
print(loglik_val)

In [83]:
# Test G=3

G = 3
itmax = 20
x = data
n = len(x)
p1 = 10 / n
p2 = 20 / n
p3 = 1 - p1 - p2
cluster_1 = x[:10]
cluster_2 = x[10:30]
cluster_3 = x[30:]
mu1 = np.mean(cluster_1, axis=0)
mu2 = np.mean(cluster_2, axis=0)
mu3 = np.mean(cluster_3, axis=0)

cov_matrix_1 = np.sum([(x[0] - mu1).reshape(-1, 1) @ (x[0] - mu1).reshape(-1, 1).T for i in cluster_1], axis=0)
cov_matrix_2 = np.sum([(x[0] - mu2).reshape(-1, 1) @ (x[0] - mu2).reshape(-1, 1).T for i in cluster_2], axis=0)
cov_matrix_3 = np.sum([(x[0] - mu3).reshape(-1, 1) @ (x[0] - mu3).reshape(-1, 1).T for i in cluster_3], axis=0)

# 2x2 matrix
cov_matrix = (cov_matrix_1 + cov_matrix_2 + cov_matrix_3) / n
print(cov_matrix)

[[2.20359809e-02 9.69443185e-01]
 [9.69443185e-01 6.62321387e+01]]


In [ ]:
prob, mean, Sigma, loglik_val = myEM()

print("prob")
print(prob)

print("mean")
print(mean)

print("Sigma")
print(Sigma)

print("loglik_val")
print(loglik_val)